<a href="https://colab.research.google.com/github/suafle/Clase2/blob/master/Actividad_4_Extragal%C3%A1ctico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EI2001 : Ciencia de Datos en Astronomía
**Profesores:** Pía Cortés, Daniela Barrientos, Matías Mattamala, Matías Suazo
##Actividad Clase 4 - Extragaláctico
###Objetivos:
1. Utilizar el paquete **astropy** para trabajar con unidades de medidas, coordenadas y archivos fits.
2. Utilizar datos del google drive personal de cada uno.
3. Lectura de archivos *fits* en objetos extragalácticos en forma de cubo de datos.
4. Utilizar el paquete **astroquery** para hacer query de objetos del SDSS y revisar *templates* de objetos conocidos.

#Sistema de coordenadas

Un sistema de coordenadas es un conjunto de convenciones establecidas con el fin de determinar las posiciones de un cuerpo en el espacio. Existen muchos sistemas de coordenadas en astronomía, algunos ejemplos son coordenadas galácticas, coordenadas eclípticas, coordenadas ecuatoriales y coordenadas horizontales. El sistema que revisaremos hoy dia es el **sistema de coordenadas ecuatorial.**

##Coordenadas ecuatoriales

Para entender un poco como funcionan los sistemas de coordenadas en astronomía, tomaremos como ejemplo las coordenadas esféricas. 

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/240px-3D_Spherical.svg.png) 

Como debiesen saber, en coordenadas esféricas la posición de una partícula queda determinada por la distancia al objeto y dos ángulos llamados ángulo azimutal y ángulo cenital, el primer ángulo recorre el plano x-y partiendo de la semirrecta x-positivo y se desplaza en sentido antihorario, mientras que el segundo ángulo parte en la recta z-positiva y se desplaza hacia el plano x-y. Inspirados en este sistema de coordenas y el hecho de que la tierra es **esférica**, resulta natural describir las estrellas y cualquier objeto en el cielo mediante estas coordenadas, sin embargo, la distancia a un determinado objeto no es algo que se pueda determinar a priori. 

Las cordenadas ecuatoriales se basan en este sistema de coordenadas, en el cual se defina la posición de los objetos solo con un par de ángulos, los cuales son la **ascención recta** y la **declinación**. La ascención recta vendría a ser un equivalente al ángulo azimutal, pues parte de eje-x positivo y se desplaza en el plano x-y en sentido antihorario. La declinación a diferencia del ángulo cenital parte del plano x-y, desplazándose hacia el eje z a diferencia del ángulo cenital. Si bien ya definimos la ascención recta (**AR**,$\alpha$) y la declinación (**DEC**,$\delta$), falta definir las ejes y los planos utilizados en el espacio real.

El plano X-Y está definido por el ecuador celeste, el cual es la proyección de la línea del ecuador. Mientras que el eje Z se encuetra definido por el eje de rotación de la Tierra.

![](http://ircamera.as.arizona.edu/astr_250/images/cel_sphere.jpg)

El eje X está definido por la recta que parte en el centro de la Tierra y se propaga hasta el punto Aries (o punto Vernal), el cual corresponde a la posición del Sol para el equinoccio de primavera (en el hemisferio norte).  Por razones histórica a veces la unidad de medida de alguno de estos ángulo puede estar en horas, lo cual corresponde a 15 grados.

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSW4PNLfsDuYAVIiZGtDBVLWxOVMZTL1IDYalaitBV2NULlAJY4Ww)

Ya habiendo entendido esto procederemos a revisar un poco sobre como trabajar con coordenadas en astropy. Primero revisaremos las unidades de medidas.


In [19]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy import coordinates as coords

# el importar la funcion units como u permite crear variables con unidades al multiplicarse por este valor
# en http://docs.astropy.org/en/stable/units/ pueden encontrar la lista con todas las posibles unidades
desplazamiento = 1 * u.m
tiempo = 2 * u.s
velocidad = desplazamiento/tiempo
print desplazamiento
print tiempo
print velocidad

1.0 m
2.0 s
0.5 m / s


Ahora revisaremos la clase SkyCoord. 

In [25]:
a = SkyCoord(10,10,frame='icrs',unit='deg')
print type(a)
print type(coords.SkyCoord(10,10,frame='icrs',unit='deg'))


<class 'astropy.coordinates.sky_coordinate.SkyCoord'>
<class 'astropy.coordinates.sky_coordinate.SkyCoord'>


#Cubo de datos

Como recuerdan de la semana pasada, los archivos se guardan en un formato llamado *fits*, debido a lo conveniente de su tamaño. Hoy a diferencia de la clase pasada trabajaremos con *fits* cubo de datos. En principio un fits es un arreglo bidimensional representado por coordenadas espaciales, en la cual se le asigna un valor a cada elemento de este arreglo bidimensional, lo cual suele ser el flujo, intensidad, número de cuentas, etc. En un cubo de datos extendemos la componente bidimensional a una tridimensional la cual puede representar alguna variable física.

En los siguientes videos se muestra como funciona un cubo de datos en caso de que esta tercera variable sea la longitud de onda.

[![MUSE datacubes](https://i.imgur.com/2feyFlG.png)](https://youtu.be/rNbdDKuDMoY?t=147)
[![MUSE datacubes](http://cdn.eso.org/videos/videoframe/eso1407d.jpg)](https://www.youtube.com/watch?v=ghVv74y_d1Q)

Como estos archivos son fits, se trabajan con astropy de la misma manera que se vio la clase pasada. Pero primero necesitamos revisar otra función antes de ponernos a trabajar con los fits.

#Montar Google Drive

En las clases anteriores hemos trabajado con datos descargados de github en tiempo real, sin embargo, esto ha sido posible porque esos archivos has sido livianos, sin embargo, en los cubos de datos suelen ser algo más pesados. Por esto a veces es necesario guardar información en nuestro **Google Drive**.

In [26]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [29]:
#Con los simples comandos anteriores ya podemos acceder a todos los archivos que estén en nuestro drive
from astropy.io import fits
filename = ('/content/gdrive/My Drive/Colab Notebooks/NGC5406.V500.rscube.fits.gz') #basta con el nombre
hdu = fits.open(filename)[0] #abre los fits, pueden abrirse comprimidos en gz
header = hdu.header
header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                  -32 / number of bits per data pixel                  
NAXIS   =                    3 / number of data axes                            
NAXIS1  =                   77 / length of data axis 1                          
NAXIS2  =                   72 / length of data axis 2                          
NAXIS3  =                 1877                                                  
EXTEND  =                    T / FITS dataset may contain extensions            
HIERARCH V500 PPAK P1 LONGSTRN = 'OGIP 1.0' / The HEASARC Long String Convention
HIERARCH V500 PPAK P1 PVERSION = '1.0     ' / PMAS fitsheader version           
HIERARCH V500 PPAK P1 FILENAME = 'run219_00120a.fits' / original filename       
HIERARCH V500 PPAK P1 DATE = '2012-05-16T20:43:01' / file creation              
HIERARCH V500 PPAK P1 OBSERVER = 'Sengupta -- Sanchez'                          
HIERARCH V500 PPAK P1 OBJECT


#Surveys Extragalácticos:

Existen muchos *surveys* que incluyen observaciones de galaxias (tanto fotometría como espectroscopía). El [Sloan Digital Sky Survey (SDSS)](https://www.sdss.org) se ha caracterizado por ser unos de los más importantes contando con 15 Data Release a la fecha. Sin embargo para el ejercicio de hoy trabajaremos con datos de [Calar Alto Legacy Integral Field Area Survey (CALIFA)](http://www.caha.es/CALIFA/public_html/?q=node/1) el cual consiste en un survey de más de 600 galaxias aplicando la técnica de *Integral Field Spectroscopy* (IFS). 
